Here I tried the airline sentiment database again with a convolutional netfwork after the embedding layer
instead of lstm.
LINK:
https://data.world/crowdflower/airline-twitter-sentiment
this database ocntains neutral, positive and negative sentiments regarding an airline in tweets
important columns: airline_sentiment, text

In [44]:
# Basic packages
import pandas as pd 
import numpy as np
import re
import collections
import matplotlib.pyplot as plt

# Packages for data preparation
from keras import optimizers
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import one_hot
from keras.preprocessing import sequence 
from keras.datasets import imdb 
from keras.models import Sequential
from keras.layers import Dense, Activation ,Dropout, Embedding , Flatten, LSTM, SpatialDropout1D,Convolution1D
from keras.callbacks import Callback
from keras.callbacks import EarlyStopping
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam

# Packages for modeling
from keras import models
from keras import layers
from keras import regularizers




NB_START_EPOCHS = 20  # Number of epochs we usually start to train with
BATCH_SIZE = 16  # Size of the batches used in the mini-batch gradient descent
PADDED_SIZE = 30

In [20]:
df = pd.read_csv('Tweets.csv', encoding = "ISO-8859-1")
df = df.reindex(np.random.permutation(df.index))  
df = df[['text', 'airline_sentiment']]


# remove neutral sentiment if needed
#df = df[df.sentiment != "neutral"]

# removing stop words like 'the, a, etc' but whitelisting ones that could be useful for sentiment
# analysis (not, no) could indicate negative emotions for example
def remove_stopwords(input_text):
        stopwords_list = stopwords.words('english')
        # Some words which might indicate a certain sentiment are kept via a whitelist
        whitelist = ["n't", "not", "no"]
        words = input_text.split() 
        clean_words = [word for word in words if (word not in stopwords_list or word in whitelist) and len(word) > 1] 
        return " ".join(clean_words) 
    
def remove_mentions(input_text):
        return re.sub(r'@\w+', '', input_text)
       
df.text = df.text.apply(remove_stopwords).apply(remove_mentions)


docs_string = df.text #df.values[:,0]#df['text']#df.values[:,0]
sentiments_string = df.airline_sentiment#df.values[:,1]#df.airline_sentiment
print("len of docs: " + str(len(docs_string)))


df.head()



len of docs: 14640


,text,airline_sentiment
10723,Now next plane broken seat! My row! Another h...,negative
8878,Airways Short Interest Down 3.5% January (JBL...,neutral
12059,"flight got Cancelled Flightled GRK DFW, LEX t...",neutral
1256,already did.,neutral
4229,"I'm not person, I've sitting Denver three hou...",negative


In [21]:
# using pre trained embedding layer
t = Tokenizer()
t.fit_on_texts(docs_string)
#calculate the word dictionary size, this is what we one-hot our data by
NB_WORDS = len(t.word_index) + 1
print("NB_WORDS: " + str(NB_WORDS))
embeddings_index = dict()
f = open('glove.6B.100d.txt', encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((NB_WORDS, 100))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

NB_WORDS: 14932
Loaded 400000 word vectors.


In [22]:
docs_encoded = [one_hot(d,
                        NB_WORDS,
                        filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                        lower=True) for d in docs_string]
docs_encoded = np.asarray(docs_encoded)
docs_encoded.shape
print("max length of docs is:")
print(len(max(docs_encoded, key=len)))
    
X = sequence.pad_sequences(docs_encoded, maxlen=PADDED_SIZE)
# Encode labels to first numbers than one-hot
le = LabelEncoder()

sentiments_le = le.fit_transform(sentiments_string)
Y = to_categorical(sentiments_le)
#get number of output classes, this is how many nodes the output layer will require
class_num = len(le.classes_)
X_train, X_valid_test, Y_train, Y_valid_test = train_test_split(X,Y, test_size = 0.3, random_state = 37)
X_valid, X_test, Y_valid, Y_test = train_test_split(X_valid_test,Y_valid_test, test_size = 0.3, random_state = 37)
print("shape of X_train")
print(X_train.shape)
print("shape of Y_train")
print(Y_train.shape)
print("shape of X_valid")
print(X_valid.shape)
print("shape of Y_valid")
print(Y_valid.shape)
print("shape of X_test")
print(X_test.shape)
print("shape of Y_test")
print(Y_test.shape)

max length of docs is:
26
shape of X_train
(10248, 30)
shape of Y_train
(10248, 3)
shape of X_valid
(3074, 30)
shape of Y_valid
(3074, 3)
shape of X_test
(1318, 30)
shape of Y_test
(1318, 3)


# Callbacks

In [66]:
patience=15
early_stopping=EarlyStopping(patience=patience, verbose=1)
checkpointer=ModelCheckpoint(filepath='weights.hdf5', save_best_only=True, verbose=1)
checkpointer_LSTM=ModelCheckpoint(filepath='weights_LSTM.hdf5', save_best_only=True, verbose=1)
checkpointer_LSTM_glove=ModelCheckpoint(filepath='weights_LSTM_glove.hdf5', save_best_only=True, verbose=1)
checkpointer_CNN=ModelCheckpoint(filepath='weights_CNN.hdf5', save_best_only=True, verbose=1)
checkpointer_CNN_glove=ModelCheckpoint(filepath='weights_CNN_glove.hdf5', save_best_only=True, verbose=1)

#TrainingHistory to get training logs from callback
class TrainingHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.valid_losses = []
        self.accs = [] 
        self.valid_accs = []
        self.epoch = 0
    
    # After every epoch append new values to the log arrays
    def on_epoch_end(self, epoch, logs={}):
        self.losses.append(logs.get('loss'))
        self.valid_losses.append(logs.get('val_loss'))
        self.accs.append(logs.get('acc'))
        self.valid_accs.append(logs.get('val_acc'))
        self.epoch += 1
            
history = TrainingHistory()

# LSTM Model without GloVe

In [46]:

embedding_vector_length = 100 
model = Sequential() 
model.add(Embedding(NB_WORDS, embedding_vector_length, input_length=PADDED_SIZE)) 
model.add(SpatialDropout1D(0.5))
model.add(LSTM(200, dropout=0.4, recurrent_dropout=0.4))
#model.add(Flatten())
model.add(Dense(class_num, activation='softmax')) 
model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy']) 
print(model.summary())
model.fit(X_train,
          Y_train,
          validation_data=(X_valid, Y_valid),
          epochs=15,
          batch_size=BATCH_SIZE,
         callbacks=[checkpointer_LSTM, early_stopping, history])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_26 (Embedding)     (None, 30, 100)           1493200   
_________________________________________________________________
spatial_dropout1d_22 (Spatia (None, 30, 100)           0         
_________________________________________________________________
lstm_26 (LSTM)               (None, 200)               240800    
_________________________________________________________________
dense_30 (Dense)             (None, 3)                 603       
Total params: 1,734,603
Trainable params: 1,734,603
Non-trainable params: 0
_________________________________________________________________
None
Train on 10248 samples, validate on 3074 samples
Epoch 1/15
10248/10248 [==============================] - 43s 4ms/step - loss: 0.7157 - acc: 0.7032 - val_loss: 0.5624 - val_acc: 0.7729

Epoch 00001: val_loss did not improve from 0.52660
Epoch 2/15
10248/1024

# LSTM Model with GloVE (pre trained embedding layer)

In [53]:
# Build the model with GloVe
model = Sequential() 
e = Embedding(NB_WORDS, 100, weights=[embedding_matrix], input_length=PADDED_SIZE)#, trainable=False)
model.add(e)
model.add(SpatialDropout1D(0.5))
model.add(LSTM(200, dropout=0.4, recurrent_dropout=0.4)) 
#model.add(Flatten())
model.add(Dense(class_num, activation='softmax')) 
model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy']) 
#model.compile(loss='categorical_crossentropy',optimizer=optimizers.SGD(lr=1e-2, momentum=0.9, nesterov=True), metrics=['accuracy'])
print(model.summary())
model.fit(X_train,
          Y_train,
          validation_data=(X_valid, Y_valid),
          epochs=15,
          batch_size=BATCH_SIZE,
         callbacks=[checkpointer_LSTM_glove, early_stopping, history]) 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_32 (Embedding)     (None, 30, 100)           1493200   
_________________________________________________________________
spatial_dropout1d_28 (Spatia (None, 30, 100)           0         
_________________________________________________________________
lstm_32 (LSTM)               (None, 200)               240800    
_________________________________________________________________
dense_36 (Dense)             (None, 3)                 603       
Total params: 1,734,603
Trainable params: 1,734,603
Non-trainable params: 0
_________________________________________________________________
None
Train on 10248 samples, validate on 3074 samples
Epoch 1/15
10248/10248 [==============================] - 36s 4ms/step - loss: 0.8208 - acc: 0.6503 - val_loss: 0.7061 - val_acc: 0.7121

Epoch 00001: val_loss did not improve from 0.68865
Epoch 2/15
10248/1024

# CNN Model without GloVe
started with adam but kept having overfitting right from the start, read that i can be
because of high learning rate and exploding gradients so i switchd to sgd to try it out

In [62]:
# Build the model with self trained embedding layer
embedding_vector_length = 100 
model = Sequential() 
model.add(Embedding(NB_WORDS, embedding_vector_length, input_length=PADDED_SIZE)) 
model.add(Convolution1D(128, 2, padding='same'))
model.add(Convolution1D(64, 2, padding='same'))
model.add(Convolution1D(32, 2, padding='same'))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(180,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(class_num, activation='softmax')) 
model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy']) 
#model.compile(loss='categorical_crossentropy',optimizer=optimizers.SGD(lr=1e-2, momentum=0.7, nesterov=True), metrics=['accuracy'])
print(model.summary())
model.fit(X_train,
          Y_train,
          validation_data=(X_valid, Y_valid),
          epochs=15,
          batch_size=BATCH_SIZE,
         callbacks=[checkpointer_CNN, early_stopping, history]) 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_41 (Embedding)     (None, 30, 100)           1493200   
_________________________________________________________________
conv1d_39 (Conv1D)           (None, 30, 128)           25728     
_________________________________________________________________
conv1d_40 (Conv1D)           (None, 30, 64)            16448     
_________________________________________________________________
conv1d_41 (Conv1D)           (None, 30, 32)            4128      
_________________________________________________________________
conv1d_42 (Conv1D)           (None, 30, 16)            1040      
_________________________________________________________________
flatten_9 (Flatten)          (None, 480)               0         
_________________________________________________________________
dropout_23 (Dropout)         (None, 480)               0         
__________

# CNN Model with GloVE

In [72]:
# Build the model with self trained embedding layer

model = Sequential() 
e = Embedding(NB_WORDS, 100, weights=[embedding_matrix], input_length=PADDED_SIZE)#, trainable=False)
model.add(e)
model.add(Convolution1D(32, 2, padding='same'))
model.add(Convolution1D(16, 2, padding='same'))
model.add(Convolution1D(8, 2, padding='same'))

model.add(Flatten())
#model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2)) 
model.add(Dropout(0.2))
model.add(Dense(180,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(class_num, activation='softmax')) 
model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy']) 
#model.compile(loss='categorical_crossentropy',optimizer=optimizers.SGD(lr=5e-3, momentum=0.9, nesterov=True), metrics=['accuracy'])
print(model.summary())
model.fit(X_train,
          Y_train,
          validation_data=(X_valid, Y_valid),
          epochs=15,
          batch_size=BATCH_SIZE,
         callbacks=[checkpointer_CNN_glove, early_stopping, history])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_49 (Embedding)     (None, 30, 100)           1493200   
_________________________________________________________________
conv1d_68 (Conv1D)           (None, 30, 32)            6432      
_________________________________________________________________
conv1d_69 (Conv1D)           (None, 30, 16)            1040      
_________________________________________________________________
conv1d_70 (Conv1D)           (None, 30, 8)             264       
_________________________________________________________________
flatten_14 (Flatten)         (None, 240)               0         
_________________________________________________________________
dropout_39 (Dropout)         (None, 240)               0         
_________________________________________________________________
dense_66 (Dense)             (None, 180)               43380     
__________

In [73]:
#evaluation test data
print("LSTM:")
model = load_model("weights_LSTM.hdf5")
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = BATCH_SIZE)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))
print("--------------------------------")

print("LSTM_glove:")
model = load_model("weights_LSTM_glove.hdf5")
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = BATCH_SIZE)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))
print("--------------------------------")

print("CNN:")
model = load_model("weights_CNN.hdf5")
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = BATCH_SIZE)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))
print("--------------------------------")

print("CNN_glove:")
model = load_model("weights_CNN_glove.hdf5")
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = BATCH_SIZE)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))
print("--------------------------------")

LSTM:
score: 0.54
acc: 0.78
--------------------------------
LSTM_glove:
score: 0.57
acc: 0.77
--------------------------------
CNN:
score: 0.59
acc: 0.77
--------------------------------
CNN_glove:
score: 0.61
acc: 0.76
--------------------------------


# Test strings you find on Twitter
Here I tested some tweets I gathered manually from the #AmericanAirlines mentions, here is the link: https://twitter.com/hashtag/americanairlines


In [65]:
model = load_model("weights_LSTM_glove.hdf5")
docs = ["lost my luggage in AUGUST. They called YESTERDAY to say it was found but was quarantined b/c it was contaminated with bugs in their storage. They said they would be delivering it to our house.",
       "So, leaving Florida at 10:50pm tonight ( supposedly) getting into Charlotte only to be put in a hotel ( which they better pay for) and flying out at 7:51am. #AmericanAirlines SUCKS once again",
       "how does #americanairlines cancel my flight and they refuse to refund me? @AmericanAir Worst customer service. Been trying for a week to get a refund.",
       "Great night at the miamiheat game hanging with my great friend @Lundy2010 Thank you @dwyanewade for dropping by for a picture",
       "Guys, my plane is going to take off into the sunrise and I’ve never been so happy! ",
       "#AmericanAirlines clearly cares way more about keeping my money than about actually serving customers.",
       "Flight diverted for a medical emergency 2 rows behind me.  Good job to the crew for acting quickly on flight 2673 from Orlando. #americanairlines @AmericanAir",
       "Great job by Cindy Stone ticketing agent at PDX. Getting me routed to St Louis hours earlier   thanks for great customer service. #AmericanAirlines @AmericanAir",
       ""]
docs_test = []
for text in docs:
    docs_test.append(remove_stopwords(text))

docs_encoded_test = [one_hot(d,
                        NB_WORDS,
                        filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                        lower=True) for d in docs_test]
docs_encoded_test = np.asarray(docs_encoded_test)

print(le.classes_)
classes = le.classes_
#for c in classes:
#    print(c)
#    print(df[ df['sentiment'] == c].size)
#    print("------------------------------")
    
X_new = sequence.pad_sequences(docs_encoded_test, maxlen=PADDED_SIZE)

results = model.predict(X_new,batch_size=1,verbose=2)
i = 0
for result in results:
    print(result)
    max_index = np.argmax(result)
    print("tweet was: ")
    print(docs[i])
    print("the tweett evaluated to: " + str(classes[max_index]))
    print("---------------------------------------------")
    i = i + 1

['negative' 'neutral' 'positive']
[0.90746015 0.05815511 0.03438473]
tweet was: 
lost my luggage in AUGUST. They called YESTERDAY to say it was found but was quarantined b/c it was contaminated with bugs in their storage. They said they would be delivering it to our house.
the tweett evaluated to: negative
---------------------------------------------
[0.5718236  0.19081207 0.23736438]
tweet was: 
So, leaving Florida at 10:50pm tonight ( supposedly) getting into Charlotte only to be put in a hotel ( which they better pay for) and flying out at 7:51am. #AmericanAirlines SUCKS once again
the tweett evaluated to: negative
---------------------------------------------
[0.9957956  0.00307553 0.00112893]
tweet was: 
how does #americanairlines cancel my flight and they refuse to refund me? @AmericanAir Worst customer service. Been trying for a week to get a refund.
the tweett evaluated to: negative
---------------------------------------------
[5.9922837e-04 6.7070345e-03 9.9269378e-01]
tweet